In [1]:
%cd /home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/trigger-detection/BGN-ST

/home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/trigger-detection/BGN-ST


In [2]:
import torch
import pickle
from dataloaders import get_data_loaders
import matplotlib.pyplot as plt
from utils.log import load_checkpoint
from collections import defaultdict
from itertools import islice
from tqdm.notebook import tqdm
from models.Bipartite_Attention_gLRI2d import Bipartite_Attention as Model
from models.Bipartite_Attention_gLRI2d import get_approximate_radii
import numpy as np
import matplotlib.colors as colors
from sklearn.cluster import KMeans, OPTICS
from scipy.stats import iqr
from numpy.linalg import inv

In [3]:
# α=0.3
config_file_path = 'train_results/biatt-glri-2d/experiment_2023-05-05_01:17:41/config.pkl'
with open(config_file_path, 'rb') as f:
    config = pickle.load(f)

In [4]:
config['lri']['sigma']

0.0008725235536466964

In [5]:
config['lri']['beta']

0.007643936757938985

In [6]:
config['model']

{'aggregator_activation': 'softmax',
 'bn': True,
 'hidden_activation': 'PReLU',
 'ln': True,
 'num_classes': 2,
 'num_features': 32,
 'recalculate_hits_mean': True,
 'self_split': False,
 'use_cholesky': False,
 'layers_spec': [[64, 8], [64, 8], [64, 8], [64, 8]]}

In [7]:
config['data']['use_center'] = True

In [8]:
dconfig = config['data']
dconfig['use_trigger'] = True
dconfig['use_nontrigger'] = False
train_data, val_data, test_data = get_data_loaders(**dconfig)

In [9]:
DEVICE = 'cuda:1'
mconfig = config['model']
model = Model(**mconfig)
model = model.to(DEVICE)

In [10]:
def get_n_hits(X):
    hits = X[..., :15].reshape(X.shape[0], X.shape[1], 5, 3)
    good_hits_mask = torch.any(hits != 0, dim=-1)
    return good_hits_mask, torch.sum(good_hits_mask, dim=-1)

In [11]:
N_BATCHES = 100
for batch in tqdm(islice(val_data, 0, N_BATCHES), total=N_BATCHES):
    mask = torch.zeros(batch.track_vector.shape[:-1])
    n = batch.n_tracks[0]
    for i, n in enumerate(batch.n_tracks):
        mask[i, :n] = 1
        
    mask = mask.to(DEVICE)
    track_vector = batch.track_vector.to(DEVICE)
    print(track_vector.shape)
    good_hits_mask, n_hits = get_n_hits(track_vector)
    e_v = torch.zeros((batch.track_vector.shape[0], batch.track_vector.shape[1], 15)).to(DEVICE)
    radii, center = get_approximate_radii(track_vector[..., :15], n_hits, good_hits_mask, False)
    print(track_vector[0, :n, 28])
    print(radii[0, :n])
    print(track_vector[0, :n, 29:31])
    print(center[0, :n])
    break

  0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([512, 47, 32])
tensor([1.7865, 3.0729, 3.2968, 1.3612, 0.7063, 0.9043, 3.2469, 1.9044, 2.2479],
       device='cuda:1')
tensor([1.7865, 3.0729, 3.2968, 1.3612, 0.7063, 0.9043, 3.2469, 1.9044, 2.2479],
       device='cuda:1')
tensor([[-177.0583,   27.2268],
        [-301.9870,  -56.8310],
        [-178.8676,  276.9480],
        [-135.3499,   14.3419],
        [  34.3570,   61.6838],
        [  18.0757,   88.6076],
        [ 281.0312, -162.6324],
        [   3.2940, -190.4112],
        [-173.0253, -143.4913]], device='cuda:1')
tensor([[-177.0584,   27.2268],
        [-301.9881,  -56.8312],
        [-178.8660,  276.9455],
        [-135.3499,   14.3419],
        [  34.3569,   61.6836],
        [  18.0757,   88.6076],
        [ 281.0330, -162.6335],
        [   3.2940, -190.4112],
        [-173.0255, -143.4915]], device='cuda:1')
